In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.6 MB/s eta 0:00:00


In [10]:
!pip install jupyter_bokeh

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.3 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.9
    Uninstalling widgetsnbextension-3.6.9:
      Successfully uninstalled widgetsnbextension-3.6.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [2]:
import os


In [3]:
import openai

In [4]:
openai.api_key  = 'sk-...'



In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [8]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role': 'user', 'content': f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))

    # Use pn.pane.HTML instead of pn.pane.Markdown and apply CSS styling
    panels.append(
        pn.Row('Assistant:', pn.pane.HTML(f'<div style="background-color: #F6F6F6; width: 600px;">{response}</div>')))

    return pn.Column(*panels)


In [11]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect interests for AI and Generative AI coaching and mentoring. \
You always respond in first person as 'we' when talking about the company. \
Your task is to give helpful answers to the user's questions. You always keep your answers small and concise and break down the large answers into small and easy-to-read paragraphs \
and bullet points. You are well versed in writing markdown format to make the response easy to read. \
You should answer only from the information given below. If you cannot answer from the information, say exactly 'I am not sure, can you rephrase the question.' \
and stop saying anything after that. Refuse to answer any question not about the info below. Never break character. \
You first greet the customer, then takes the order, \
and then asks how much years of experience the customer has. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
AI + LLM courese for senior IT professionals with 10+ years of experience  $795 \
AI + LLM courese for senior IT professionals with 0 to 10 years of experience $895 \
An option to customize the course $66 per hour \
Time duration of the course would be as follows \
AI + LLM courese for senior IT professionals with 10+ years of experience 3 months or 12 weeks \
AI + LLM courese for senior IT professionals with 0 to 10 years of experience 5 months or 18 weeks \
For customized course, it depends on the content \
The prerequisite for taking these courses is Python\
"""} ]  # accumulate messages
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [12]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous order. Itemize the price for each item\
 The fields should be 1) Course 2) Duation 3) any other requirement '},
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
    "Course": "AI + LLM course for senior IT professionals with 0 to 10 years of experience",
    "Price": "$895",
    "Duration": "5 months or 18 weeks",
    "Other Requirements": "Milestone-based payment structure"
}
